<a href="https://colab.research.google.com/github/shake/colab-Llama-2-ipynb/blob/main/04_AWQ_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AWQ quantization (AutoAWQ ) for lighter and faster quantized inference of LLMs

In June 2023, the [AWQ: Activation-aware Weight Quantization for LLM Compression and Acceleration](https://arxiv.org/pdf/2306.00978.pdf) has been published by Ji Lin et al. The paper details an algorithm to compress any transformer-based language model in few bits with a tiny performance degradation. To learn more about this quantization method, Professor Song Han gives a excellent [talk](https://hanlab.mit.edu/projects/awq).

We new support loading models that are quantized with GPTQ algorithm in 🤗 transformers from two different libraries: [LLM-AWQ](https://github.com/mit-han-lab/llm-awq) and [AutoAWQ](https://github.com/casper-hansen/AutoAWQ).

Let's check in this notebook the different options (quantize a model, push a quantized model on the 🤗 Hub, load an already quantized model from the Hub, etc.) that are offered in this integration!

## check colab



AutoAWQ maybe need 3.6 transformers, colab is 3.5，need update，and restart runtime。



In [1]:
# check default cuda version
!nvcc --version 4


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
# check default torch version
import torch
print(torch.__version__)

2.1.0+cu121


In [3]:
# check default transformers version
import transformers
print (transformers.__version__)

4.35.2


In [4]:
!pip install -q -U git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [1]:
# remenber runtime--restart session
import transformers
print (transformers.__version__)

4.37.0.dev0


## AWQ

In [ ]:
!pip install -q  autoawq

In [3]:
# make a soft link for  /root/.cache
!mkdir -p /root/.cache/huggingface
!ln -s /root/.cache/huggingface /content

need to login hugging face, download meta Llama-2-7b and upload quantized models

In [4]:
# login huggingface
! git config --global credential.helper store
!huggingface-cli login --token hf_ziljuWSLzXXrlDvr --add-to-git-credential

Token is valid (permission: read).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


**quantized model**

In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_path = 'meta-llama/Llama-2-7b-hf'
quant_path = 'Llama-2-7b-awq'
quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM" }

# Load model
model = AutoAWQForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# Quantize
model.quantize(tokenizer, quant_config=quant_config)

# Save quantized model
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

In [7]:
quant_config

{'zero_point': True, 'q_group_size': 128, 'w_bit': 4, 'version': 'GEMM'}

## compatible with transformers

 In order to make it compatible with transformers, we need to modify the config file.

In [ ]:
from transformers import AwqConfig, AutoConfig
from huggingface_hub import HfApi

# modify the config file so that it is compatible with transformers integration
quantization_config = AwqConfig(
    bits=quant_config["w_bit"],
    group_size=quant_config["q_group_size"],
    zero_point=quant_config["zero_point"],
    version=quant_config["version"].lower(),
).to_dict()

# the pretrained transformers model is stored in the model attribute + we need to pass a dict
model.model.config.quantization_config = quantization_config
# a second solution would be to use Autoconfig and push to hub (what we do at llm-awq)


# save model weights
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

## upload models to Huggingface

In [ ]:
api = HfApi()
api.upload_folder(
    folder_path="Llama-2-7b-awq",
    repo_id="chenshake/Llama-2-7b-awq",
    repo_type="model",
)

## Test

Now we can use our model with transformers library to run inference !

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("chenshake/Llama-2-7b-awq")
model = AutoModelForCausalLM.from_pretrained("chenshake/Llama-2-7b-awq").to(0)

text = "Hello my name is"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = model.generate(**inputs, max_new_tokens=5)
print(tokenizer.decode(out[0], skip_special_tokens=True))

## Loading  model from huggingface on Google colab

In [ ]:
model_id = "chenshake/Llama-2-7b-awq"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda")

In [ ]:
text = "User:\nHello can you provide me with top-3 cool places to visit in Paris?\n\nAssistant:\n"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = model.generate(**inputs, max_new_tokens=300)
print(tokenizer.decode(out[0], skip_special_tokens=True))